In [57]:
import pandas as pd  # Importa la librería pandas y la asigna al alias 'pd', utilizada para la manipulación y análisis de datos.
import time  # Importa el módulo time, que proporciona diversas funciones relacionadas con la manipulación del tiempo, como delays y medición de intervalos.
import json
import numpy as np
import random
import time
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pandas import json_normalize

from selenium import webdriver  # Importa el módulo webdriver de Selenium, utilizado para automatizar navegadores web.
from selenium.webdriver.common.by import By  # Importa el módulo By, que proporciona métodos para localizar elementos en una página web por diferentes atributos (ID, nombre, clase, etc.).
from selenium.webdriver.support import expected_conditions as EC  # Importa el módulo expected_conditions, que contiene una serie de condiciones que se pueden esperar (esperar a que un elemento sea clicable, visible, etc.).
from selenium.webdriver.support.ui import WebDriverWait  # Importa el módulo WebDriverWait, que permite esperar hasta que se cumpla una condición específica.
from selenium.webdriver.chrome.service import Service  # Importa el módulo Service, que permite iniciar y controlar el servicio del navegador Chrome.
from selenium.webdriver.chrome.options import Options  # Importa el módulo Options, que permite personalizar las opciones de configuración del navegador Chrome.
from webdriver_manager.chrome import ChromeDriverManager  # Importa el ChromeDriverManager, que gestiona automáticamente la descarga e instalación del controlador de Chrome.
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
# Configuración
import warnings  # Importa el módulo warnings, que maneja los mensajes de advertencia en Python.
warnings.filterwarnings("ignore")  # Configura para ignorar todas las advertencias.
warnings.simplefilter(action='ignore', category=FutureWarning)  # Configura para ignorar las advertencias de tipo FutureWarning, que se utilizan para notificar sobre cambios que pueden ocurrir en versiones futuras de las librerías.
pd.set_option('display.max_columns', None)  # Configura pandas para mostrar todas las columnas de un DataFrame al imprimirlo, sin truncar la visualización.
pd.set_option('display.max_rows', None)  # Configura pandas para mostrar todas las filas de un DataFrame al imprimirlo, sin truncar la visualización.

In [58]:
def random_sleep_time():
    return random.uniform(3, 6)

In [59]:
def setup_driver():
    driver = webdriver.Chrome()
    return driver


def close_ad_popup(driver):
    try:
        # Esperar hasta 10 segundos para que aparezca la publicidad
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "webpush-swal2-container"))
        )
        
        # Intentar cerrar con el botón 'x'
        try:
            close_button = driver.find_element(By.CLASS_NAME, "webpush-swal2-close")
            close_button.click()
            print("Publicidad cerrada exitosamente con el botón 'x'.")
            return
        except NoSuchElementException:
            print("No se encontró el botón 'x' para cerrar la publicidad.")
        
        # Si no se pudo cerrar con 'x', intentar con el botón "FREE TRIAL"
        try:
            free_trial_button = driver.find_element(By.XPATH, "//button[contains(@class, 'webpush-swal2-confirm')]")
            free_trial_button.click()
            print("Publicidad cerrada exitosamente con el botón 'FREE TRIAL'.")
            return
        except NoSuchElementException:
            print("No se encontró el botón 'FREE TRIAL'.")
        
        # Si aún no se ha cerrado, intentar hacer clic fuera de la publicidad
        try:
            overlay = driver.find_element(By.CLASS_NAME, "webpush-swal2-container")
            driver.execute_script("arguments[0].click();", overlay)
            print("Se intentó cerrar la publicidad haciendo clic fuera de ella.")
        except Exception as e:
            print(f"No se pudo cerrar la publicidad haciendo clic fuera: {str(e)}")
        
    except TimeoutException:
        print("No se encontró la publicidad o no apareció en el tiempo esperado.")
    except Exception as e:
        print(f"Error inesperado al intentar cerrar la publicidad: {str(e)}")

In [60]:
driver = webdriver.Chrome()
whoscored_url = "https://www.whoscored.com/Regions/250/Tournaments/12/Seasons/10456/Stages/23663/Show/Europe-Champions-League-2024-2025"
driver.get(whoscored_url)
# soup = BeautifulSoup(driver.page_source, 'html.parser')

In [61]:
print("Esperando que la página cargue completamente...")
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

print("Intentando cerrar la publicidad...")
close_ad_popup(driver)

Esperando que la página cargue completamente...
Intentando cerrar la publicidad...
No se encontró la publicidad o no apareció en el tiempo esperado.


In [62]:
# Wait for the specific element to be present
try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "script[data-hypernova-key='tournamentfixtures']"))
    )
except:
    print("Timed out waiting for page to load")

page_source = driver.page_source
driver.quit()

In [63]:
def extract_script_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tag = soup.find('script', attrs={'data-hypernova-key': 'tournamentfixtures'})
    
    if script_tag:
        comment_content = script_tag.string
        match = re.search(r'<!--(.+?)-->', comment_content, re.DOTALL)
        
        if match:
            return match.group(1).strip()
    
    return None

In [64]:
extracted_content = extract_script_content(page_source)

if extracted_content:
    print("Contenido extraído exitosamente.")
    print(extracted_content[:500] + "...")  # Imprime los primeros 500 caracteres
else:
    print("No se encontró el contenido del script comentado.")

Contenido extraído exitosamente.
{"createdAt":"2024-10-07T12:52:51Z","version":638635630660000000,"timeout":0,"tournaments":[{"tournamentId":12,"stageId":23663,"stageName":"Champions League","regionId":250,"tournamentName":"Champions League","seasonName":"2024/2025","seasonId":10456,"stageSortOrder":344,"sex":1,"tournamentSortOrder":220,"regionCode":"ceu","regionName":"Europe","isOpta":true,"navigationDisplayMode":2,"matches":[{"stageId":23663,"id":1866220,"status":6,"startTime":"2024-10-01T17:45:00","homeTeamId":361,"homeTeamN...


In [65]:
extracted_content

'{"createdAt":"2024-10-07T12:52:51Z","version":638635630660000000,"timeout":0,"tournaments":[{"tournamentId":12,"stageId":23663,"stageName":"Champions League","regionId":250,"tournamentName":"Champions League","seasonName":"2024/2025","seasonId":10456,"stageSortOrder":344,"sex":1,"tournamentSortOrder":220,"regionCode":"ceu","regionName":"Europe","isOpta":true,"navigationDisplayMode":2,"matches":[{"stageId":23663,"id":1866220,"status":6,"startTime":"2024-10-01T17:45:00","homeTeamId":361,"homeTeamName":"Salzburg","homeYellowCards":3,"homeRedCards":0,"awayTeamId":2332,"awayTeamName":"Brest","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 19:21:41Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"at","awayTeamCountryCode":"fr","commentCount":6,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Austria","awayTeamCountryName":"France","startTimeUtc":"2024-10-01T16

In [66]:


# Cargar el JSON
data = json.loads(extracted_content)

# DataFrame de torneos
tournaments_df = pd.DataFrame(data['tournaments'])

# DataFrame de partidos
matches_df = json_normalize(
    data['tournaments'], 
    record_path=['matches'], 
    meta=[
        ['tournamentId'], 
        ['stageName'], 
        ['tournamentName'], 
        ['seasonName']
    ],
    record_prefix='match_',
    meta_prefix='tournament_'
)

# DataFrame de incidentes (goles, tarjetas, etc.)
incidents_df = json_normalize(
    data['tournaments'], 
    record_path=['matches', 'incidents'], 
    meta=[
        ['matches', 'id'], 
        ['matches', 'homeTeamName'], 
        ['matches', 'awayTeamName'],
        ['tournamentName'],
        ['stageName']
    ],
    record_prefix='incident_',
    meta_prefix='match_'
)

# DataFrame de equipos (combinando información de equipos locales y visitantes)
home_teams = matches_df[['match_id', 'match_homeTeamId', 'match_homeTeamName', 'match_homeTeamCountryCode', 'match_homeTeamCountryName']]
away_teams = matches_df[['match_id', 'match_awayTeamId', 'match_awayTeamName', 'match_awayTeamCountryCode', 'match_awayTeamCountryName']]

home_teams.columns = ['matchId', 'teamId', 'teamName', 'countryCode', 'countryName']
away_teams.columns = ['matchId', 'teamId', 'teamName', 'countryCode', 'countryName']

teams_df = pd.concat([home_teams, away_teams]).drop_duplicates(subset='teamId').reset_index(drop=True)

# DataFrame de localización
localization_df = pd.DataFrame([data['localization']])

# DataFrame de meta
meta_df = pd.DataFrame([data['meta']])

# Imprimir información sobre los DataFrames creados
print("DataFrames creados:")
print(f"1. Torneos: {tournaments_df.shape[0]} filas, {tournaments_df.shape[1]} columnas")
print(f"2. Partidos: {matches_df.shape[0]} filas, {matches_df.shape[1]} columnas")
print(f"3. Incidentes: {incidents_df.shape[0]} filas, {incidents_df.shape[1]} columnas")
print(f"4. Equipos: {teams_df.shape[0]} filas, {teams_df.shape[1]} columnas")
print(f"5. Localización: {localization_df.shape[0]} filas, {localization_df.shape[1]} columnas")
print(f"6. Meta: {meta_df.shape[0]} filas, {meta_df.shape[1]} columnas")



DataFrames creados:
1. Torneos: 1 filas, 15 columnas
2. Partidos: 18 filas, 46 columnas
3. Incidentes: 59 filas, 15 columnas
4. Equipos: 36 filas, 5 columnas
5. Localización: 1 filas, 48 columnas
6. Meta: 1 filas, 1 columnas


In [67]:
print("\nTorneos:")
tournaments_df.head()


Torneos:


,tournamentId,stageId,stageName,regionId,tournamentName,seasonName,seasonId,stageSortOrder,sex,tournamentSortOrder,regionCode,regionName,isOpta,navigationDisplayMode,matches
0,12,23663,Champions League,250,Champions League,2024/2025,10456,344,1,220,ceu,Europe,True,2,"[{'stageId': 23663, 'id': 1866220, 'status': 6..."


In [68]:
print("\nPartidos:" + str(len(matches_df)))
matches_df.head()


Partidos:18


,match_stageId,match_id,match_status,match_startTime,match_homeTeamId,match_homeTeamName,match_homeYellowCards,match_homeRedCards,match_awayTeamId,match_awayTeamName,match_awayYellowCards,match_awayRedCards,match_hasIncidentsSummary,match_hasPreview,match_scoreChangedAt,match_elapsed,match_lastScorer,match_isTopMatch,match_homeTeamCountryCode,match_awayTeamCountryCode,match_commentCount,match_isLineupConfirmed,match_isStreamAvailable,match_matchIsOpta,match_homeTeamCountryName,match_awayTeamCountryName,match_startTimeUtc,match_homeScore,match_awayScore,match_incidents,match_bets,match_aggregateWinnerField,match_winnerField,match_period,match_extraResultField,match_homeExtratimeScore,match_awayExtratimeScore,match_homePenaltyScore,match_awayPenaltyScore,match_startedAtUtc,match_firstHalfEndedAtUtc,match_secondHalfStartedAtUtc,tournament_tournamentId,tournament_stageName,tournament_tournamentName,tournament_seasonName
0,23663,1866220,6,2024-10-01T17:45:00,361,Salzburg,3,0,2332,Brest,2,0,True,True,2024-10-01 19:21:41Z,FT,1,False,at,fr,6,True,False,False,Austria,France,2024-10-01T16:45:00Z,0,4,"[{'minute': '24', 'type': 1, 'subType': 1, 'pl...",None,None,1.0,7,None,None,None,None,None,2024-10-01T16:45:29Z,2024-10-01T17:35:34Z,2024-10-01T17:50:47Z,12,Champions League,Champions League,2024/2025
1,23663,1866237,6,2024-10-01T17:45:00,41,VfB Stuttgart,1,0,354,Sparta Prague,1,0,True,True,2024-10-01 18:18:09Z,FT,1,False,de,cz,15,True,False,False,Germany,Czech Republic,2024-10-01T16:45:00Z,1,1,"[{'minute': '7', 'type': 1, 'subType': 1, 'pla...",None,None,NaN,7,None,None,None,None,None,2024-10-01T16:45:49Z,2024-10-01T17:30:54Z,2024-10-01T17:46:44Z,12,Champions League,Champions League,2024/2025
2,23663,1866209,6,2024-10-01T20:00:00,129,PSV Eindhoven,3,0,296,Sporting CP,2,0,True,True,2024-10-01 21:43:19Z,FT,1,True,nl,pt,7,True,False,False,Netherlands,Portugal,2024-10-01T19:00:00Z,1,1,"[{'minute': '15', 'type': 1, 'subType': 1, 'pl...",None,None,NaN,7,None,None,None,None,None,2024-10-01T19:00:25Z,2024-10-01T19:48:21Z,2024-10-01T20:05:05Z,12,Champions League,Champions League,2024/2025
3,23663,1866154,6,2024-10-01T20:00:00,36,Bayer Leverkusen,2,0,80,AC Milan,3,0,True,True,2024-10-01 21:10:06Z,FT,0,True,de,it,8,True,False,False,Germany,Italy,2024-10-01T19:00:00Z,1,0,"[{'minute': '51', 'type': 1, 'subType': 1, 'pl...",None,None,0.0,7,None,None,None,None,None,2024-10-01T19:00:17Z,2024-10-01T19:47:44Z,2024-10-01T20:03:39Z,12,Champions League,Champions League,2024/2025
4,23663,1866166,6,2024-10-01T20:00:00,13,Arsenal,1,0,304,Paris Saint-Germain,1,0,True,True,2024-10-01 20:35:58Z,FT,0,True,gb-eng,fr,15,True,False,False,England,France,2024-10-01T19:00:00Z,2,0,"[{'minute': '20', 'type': 1, 'subType': 1, 'pl...",None,None,0.0,7,None,None,None,None,None,2024-10-01T19:00:28Z,2024-10-01T19:46:37Z,2024-10-01T20:02:20Z,12,Champions League,Champions League,2024/2025


In [69]:
print("\nIncidentes:")
incidents_df.head()


Incidentes:


,incident_minute,incident_type,incident_subType,incident_playerName,incident_playerId,incident_participatingPlayerName,incident_participatingPlayerId,incident_runningScore,incident_field,incident_period,match_matches.id,match_matches.homeTeamName,match_matches.awayTeamName,match_tournamentName,match_stageName
0,24,1,1,Abdallah Sima,404201,Ludovic Ajorque,234364,None,1,1,1866220,Salzburg,Brest,Champions League,Champions League
1,66,1,1,Mahdi Camara,349960,None,0,None,1,3,1866220,Salzburg,Brest,Champions League,Champions League
2,70,1,1,Abdallah Sima,404201,None,0,None,1,3,1866220,Salzburg,Brest,Champions League,Champions League
3,75,1,1,Mathias Pereira Lage,303314,Mama Baldé,345396,None,1,3,1866220,Salzburg,Brest,Champions League,Champions League
4,7,1,1,Enzo Millot,398374,Maximilian Mittelstädt,261020,None,0,1,1866237,VfB Stuttgart,Sparta Prague,Champions League,Champions League


In [75]:
print(len(teams_df))

36


In [70]:
print("\nEquipos:")
teams_df.head()


Equipos:


,matchId,teamId,teamName,countryCode,countryName
0,1866220,361,Salzburg,at,Austria
1,1866237,41,VfB Stuttgart,de,Germany
2,1866209,129,PSV Eindhoven,nl,Netherlands
3,1866154,36,Bayer Leverkusen,de,Germany
4,1866166,13,Arsenal,gb-eng,England


In [ ]:
# https://d2zywfiolv4f83.cloudfront.net/img/teams/44.png

In [76]:
import pandas as pd
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# Asumimos que ya tienes un DataFrame llamado 'teams_df' con una columna 'teamId'

def add_image_url(df):
    """Añade la URL de la imagen al DataFrame basada en el teamId."""
    base_url = "https://d2zywfiolv4f83.cloudfront.net/img/teams/"
    df['imageUrl'] = base_url + df['teamId'].astype(str) + '.png'
    return df

def download_image(row):
    """Descarga la imagen para un equipo específico."""
    team_id = row['teamId']
    url = row['imageUrl']
    filename = f"img/{team_id}.png"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Esto levantará una excepción para respuestas HTTP no exitosas
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Descargada imagen para el equipo {team_id}")
        return True
    except Exception as e:
        print(f"Error al descargar la imagen para el equipo {team_id}: {str(e)}")
        return False

def process_team_images(teams_df):
    """Procesa las imágenes de los equipos: añade URLs y descarga las imágenes."""
    # Añadir URLs de imágenes al DataFrame
    teams_df = add_image_url(teams_df)
    
    # Crear directorio para las imágenes si no existe
    if not os.path.exists('img'):
        os.makedirs('img')
    
    # Descargar imágenes usando multithreading para mayor eficiencia
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(download_image, row) for _, row in teams_df.iterrows()]
        
        for future in as_completed(futures):
            future.result()  # Esto asegurará que se impriman los mensajes de error si los hay

    print("Proceso de descarga de imágenes completado.")
    return teams_df

# Uso del código
# Asumiendo que ya tienes tu DataFrame 'teams_df'
teams_df = process_team_images(teams_df)



Descargada imagen para el equipo 36
Descargada imagen para el equipo 129
Descargada imagen para el equipo 75
Descargada imagen para el equipo 44
Descargada imagen para el equipo 41
Descargada imagen para el equipo 13
Descargada imagen para el equipo 698
Descargada imagen para el equipo 65
Descargada imagen para el equipo 361
Descargada imagen para el equipo 1706
Descargada imagen para el equipo 390
Descargada imagen para el equipo 2783
Descargada imagen para el equipo 299
Descargada imagen para el equipo 354
Descargada imagen para el equipo 26
Descargada imagen para el equipo 7614
Descargada imagen para el equipo 684
Descargada imagen para el equipo 296
Descargada imagen para el equipo 24
Descargada imagen para el equipo 607
Descargada imagen para el equipo 2332
Descargada imagen para el equipo 304
Descargada imagen para el equipo 80
Descargada imagen para el equipo 167
Descargada imagen para el equipo 256
Descargada imagen para el equipo 579
Descargada imagen para el equipo 587
Descar

,teamId,teamName,imageUrl
0,361,Salzburg,https://d2zywfiolv4f83.cloudfront.net/img/team...
1,41,VfB Stuttgart,https://d2zywfiolv4f83.cloudfront.net/img/team...
2,129,PSV Eindhoven,https://d2zywfiolv4f83.cloudfront.net/img/team...
3,36,Bayer Leverkusen,https://d2zywfiolv4f83.cloudfront.net/img/team...
4,13,Arsenal,https://d2zywfiolv4f83.cloudfront.net/img/team...


In [77]:
# Ahora teams_df tiene una nueva columna 'imageUrl' y las imágenes están descargadas en el directorio 'img'
teams_df.head()

,matchId,teamId,teamName,countryCode,countryName,imageUrl
0,1866220,361,Salzburg,at,Austria,https://d2zywfiolv4f83.cloudfront.net/img/team...
1,1866237,41,VfB Stuttgart,de,Germany,https://d2zywfiolv4f83.cloudfront.net/img/team...
2,1866209,129,PSV Eindhoven,nl,Netherlands,https://d2zywfiolv4f83.cloudfront.net/img/team...
3,1866154,36,Bayer Leverkusen,de,Germany,https://d2zywfiolv4f83.cloudfront.net/img/team...
4,1866166,13,Arsenal,gb-eng,England,https://d2zywfiolv4f83.cloudfront.net/img/team...


In [71]:
print("\nLocalización:")
localization_df.head()


Localización:


,match_status_all,match_status_live,match_status_upcoming,livescores_info_refresh,livescores_info_starts_hrs_mins,livescores_info_starts_mins,livescores_info_loading,favourite_info_please,favourite_lnk_singin,favourite_lnk_register,info_today,lbl_user_odds_display_type,sex_abbrv_woman,ls_view_in_mc,lnk_datepicker_view_current_week,lnk_datepicker_view_current_month,fltr_livescores_summary_yesterday,fltr_livescores_summary_today,fltr_livescores_summary_tomorrow,GK,DR,DC,DL,DMR,DMC,DML,MR,MC,ML,AMR,AMC,AM,AML,FWR,FW,FC,FWL,Sub,abd,aet,can,et1,et2,ft,ht,pen,post,susp
0,All,In Play,Upcoming,Refreshing in {0} seconds...,Matches start in {0} hours {1} minutes,Matches start in {0} minutes,Loading...,Please,Sign In,Sign Up,Today,Odds,W,View in Match Centre,View current week,View current month,Yesterday,Today,Tomorrow,GK,DR,DC,DL,DMR,DMC,DML,MR,MC,ML,AMR,AMC,AMC,AML,FWR,FW,FW,FWL,Sub,Abd,AET,Can,ET1,ET2,FT,HT,PEN,Post,Susp


In [72]:
print("\nMeta:")
meta_df.head()


Meta:


,imagesUrlBase
0,https://d2zywfiolv4f83.cloudfront.net/img/


In [78]:
tournaments_df.to_excel('tournaments.xlsx')
teams_df.to_excel('teams.xlsx')

In [ ]:
livescoreincidents: "/matchesfeed/{id}/IncidentsSummary/",
stagefixtures: "/tournamentsfeed/{stageId}/Fixtures/",
teamfixtures: "/teamsfeed/{teamId}/Fixtures/",
standings: "/stagesfeed/{stageId}/standings/",
forms: "/stagesfeed/{stageId}/forms/",
history: "/stagesfeed/{stageId}/history/",
streaks: "/stagesfeed/{stageId}/streaks/",
goals: "/tournamentsfeed/{stageId}/PlayerStatistics/",
cards: "/tournamentsfeed/{stageId}/PlayerStatistics/",
"team-goals": "/teamsfeed/{teamId}/PlayerStatistics/",
"team-cards": "/teamsfeed/{teamId}/PlayerStatistics/",
previousmeetings: "/teamsfeed/{homeTeamId}/PreviousMeetings/",
statistics: "/statisticsfeed/",
"side-box-statistics": "/statisticsfeed/{statsType}/SideBoxStatistics/",
regionteams: "/teamsfeed/{id}/region",
"ws-stage-stat": "/stagestatfeed/",
"ws-teams-stage-stat": "/stagestatfeed/{stageId}/stageteams/",
"ws-stage-filtered-team-stat": "/stagestatfeed/{stageId}/teamsstagefiltered/",
"ws-teams-filtered-stage-stat": "/stagestatfeed/{stageId}/stageteamsfiltered/",
"stage-top-player-stats": "/stagestatfeed/{stageId}/stagetopplayers",
"live-team-stat": "/optamatchstatfeed/",
"team-fixtures": "/teamsfeed/{teamId}/H2HFixtures/",
"match-header": "/matchesfeed/{id}/MatchHeader",
"match-live-update": "/matchesfeed/{id}/LiveMatch",
"match-commentary": "/matchesfeed/{id}/MatchCommentary",
"live-player-stats": "/matchesfeed/{id}/LivePlayerStats",
"betting-stats": "/bettingstatfeed/",
"overall-player-stat": "/stageplayerstatfeed/{playerId}/Overall",
"stage-player-stat": "/stageplayerstatfeed/",
"overall-team-stat": "/stageteamstatfeed/{teamId}/Overall",
"stage-team-stat": "/stageteamstatfeed/",
"stage-h2h-player-stat": "/stageplayerstatfeed/{stageId}/H2HTeamPlayers",
"player-tournament-stat": "/stageplayerstatfeed/{playerId}/PlayerTournamentStats",
"facts-filter": "/Facts/Data",
"player-heatmap": "/Players/{id}/Heatmap",
"match-centre": "/matchesfeed/{id}/MatchCentre",
"match-centre2": "/matchesfeed/{id}/MatchCentre2",
"player-tournament-history-stat": "/stageplayerstatfeed/{playerId}/PlayerHistoryTournamentStats",
"custom-standings": "/tournamentsfeed/{stageId}/CustomStandings/"

In [ ]:
https://seg.ad.gt/api/v2/match

In [ ]:
<script type="application/json" data-hypernova-key="tournamentfixtures" data-hypernova-id="041fc081-04ec-4f77-a511-bb83497ffc0c"><!--{"createdAt":"2024-10-05T10:43:46Z","version":638635630660000000,"timeout":0,"tournaments":[{"tournamentId":12,"stageId":23663,"stageName":"Champions League","regionId":250,"tournamentName":"Champions League","seasonName":"2024/2025","seasonId":10456,"stageSortOrder":344,"sex":1,"tournamentSortOrder":220,"regionCode":"ceu","regionName":"Europe","isOpta":true,"navigationDisplayMode":2,"matches":[{"stageId":23663,"id":1866220,"status":6,"startTime":"2024-10-01T17:45:00","homeTeamId":361,"homeTeamName":"Salzburg","homeYellowCards":3,"homeRedCards":0,"awayTeamId":2332,"awayTeamName":"Brest","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 19:21:41Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"at","awayTeamCountryCode":"fr","commentCount":6,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Austria","awayTeamCountryName":"France","startTimeUtc":"2024-10-01T16:45:00Z","homeScore":0,"awayScore":4,"incidents":[{"minute":"24","type":1,"subType":1,"playerName":"Abdallah Sima","playerId":404201,"participatingPlayerName":"Ludovic Ajorque","participatingPlayerId":234364,"runningScore":null,"field":1,"period":1},{"minute":"66","type":1,"subType":1,"playerName":"Mahdi Camara","playerId":349960,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3},{"minute":"70","type":1,"subType":1,"playerName":"Abdallah Sima","playerId":404201,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3},{"minute":"75","type":1,"subType":1,"playerName":"Mathias Pereira Lage","playerId":303314,"participatingPlayerName":"Mama Baldé","participatingPlayerId":345396,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T16:45:29Z","firstHalfEndedAtUtc":"2024-10-01T17:35:34Z","secondHalfStartedAtUtc":"2024-10-01T17:50:47Z"},{"stageId":23663,"id":1866237,"status":6,"startTime":"2024-10-01T17:45:00","homeTeamId":41,"homeTeamName":"VfB Stuttgart","homeYellowCards":1,"homeRedCards":0,"awayTeamId":354,"awayTeamName":"Sparta Prague","awayYellowCards":1,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 18:18:09Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"de","awayTeamCountryCode":"cz","commentCount":15,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Germany","awayTeamCountryName":"Czech Republic","startTimeUtc":"2024-10-01T16:45:00Z","homeScore":1,"awayScore":1,"incidents":[{"minute":"7","type":1,"subType":1,"playerName":"Enzo Millot","playerId":398374,"participatingPlayerName":"Maximilian Mittelstädt","participatingPlayerId":261020,"runningScore":null,"field":0,"period":1},{"minute":"32","type":1,"subType":1,"playerName":"Kaan Kairinen","playerId":202261,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":1}],"bets":null,"aggregateWinnerField":null,"winnerField":null,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T16:45:49Z","firstHalfEndedAtUtc":"2024-10-01T17:30:54Z","secondHalfStartedAtUtc":"2024-10-01T17:46:44Z"},{"stageId":23663,"id":1866209,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":129,"homeTeamName":"PSV Eindhoven","homeYellowCards":3,"homeRedCards":0,"awayTeamId":296,"awayTeamName":"Sporting CP","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:43:19Z","elapsed":"FT","lastScorer":1,"isTopMatch":true,"homeTeamCountryCode":"nl","awayTeamCountryCode":"pt","commentCount":7,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Netherlands","awayTeamCountryName":"Portugal","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":1,"awayScore":1,"incidents":[{"minute":"15","type":1,"subType":1,"playerName":"Jerdy Schouten","playerId":331555,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"83","type":1,"subType":1,"playerName":"Daniel Bragança","playerId":384888,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":null,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:25Z","firstHalfEndedAtUtc":"2024-10-01T19:48:21Z","secondHalfStartedAtUtc":"2024-10-01T20:05:05Z"},{"stageId":23663,"id":1866154,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":36,"homeTeamName":"Bayer Leverkusen","homeYellowCards":2,"homeRedCards":0,"awayTeamId":80,"awayTeamName":"AC Milan","awayYellowCards":3,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:10:06Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"de","awayTeamCountryCode":"it","commentCount":8,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Germany","awayTeamCountryName":"Italy","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":1,"awayScore":0,"incidents":[{"minute":"51","type":1,"subType":1,"playerName":"Victor Boniface","playerId":384000,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:17Z","firstHalfEndedAtUtc":"2024-10-01T19:47:44Z","secondHalfStartedAtUtc":"2024-10-01T20:03:39Z"},{"stageId":23663,"id":1866166,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":13,"homeTeamName":"Arsenal","homeYellowCards":1,"homeRedCards":0,"awayTeamId":304,"awayTeamName":"Paris Saint-Germain","awayYellowCards":1,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 20:35:58Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"gb-eng","awayTeamCountryCode":"fr","commentCount":15,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"England","awayTeamCountryName":"France","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":2,"awayScore":0,"incidents":[{"minute":"20","type":1,"subType":1,"playerName":"Kai Havertz","playerId":326413,"participatingPlayerName":"Leandro Trossard","participatingPlayerId":113994,"runningScore":null,"field":0,"period":1},{"minute":"35","type":1,"subType":1,"playerName":"Bukayo Saka","playerId":367185,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:28Z","firstHalfEndedAtUtc":"2024-10-01T19:46:37Z","secondHalfStartedAtUtc":"2024-10-01T20:02:20Z"},{"stageId":23663,"id":1866104,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":698,"homeTeamName":"Slovan Bratislava","homeYellowCards":0,"homeRedCards":0,"awayTeamId":167,"awayTeamName":"Manchester City","awayYellowCards":0,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:30:21Z","elapsed":"FT","lastScorer":1,"isTopMatch":true,"homeTeamCountryCode":"sk","awayTeamCountryCode":"gb-eng","commentCount":16,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Slovakia","awayTeamCountryName":"England","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":0,"awayScore":4,"incidents":[{"minute":"8","type":1,"subType":1,"playerName":"Ilkay Gündogan","playerId":77464,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":1},{"minute":"15","type":1,"subType":1,"playerName":"Phil Foden","playerId":331254,"participatingPlayerName":"Jérémy Doku","participatingPlayerId":388098,"runningScore":null,"field":1,"period":1},{"minute":"58","type":1,"subType":1,"playerName":"Erling Haaland","playerId":315227,"participatingPlayerName":"Rico Lewis","participatingPlayerId":444791,"runningScore":null,"field":1,"period":3},{"minute":"74","type":1,"subType":1,"playerName":"James McAtee","playerId":425576,"participatingPlayerName":"Phil Foden","participatingPlayerId":331254,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:39Z","firstHalfEndedAtUtc":"2024-10-01T19:45:42Z","secondHalfStartedAtUtc":"2024-10-01T20:01:12Z"},{"stageId":23663,"id":1866108,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":75,"homeTeamName":"Inter","homeYellowCards":2,"homeRedCards":0,"awayTeamId":579,"awayTeamName":"FK Crvena Zvezda","awayYellowCards":1,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:38:44Z","elapsed":"FT","lastScorer":0,"isTopMatch":false,"homeTeamCountryCode":"it","awayTeamCountryCode":"rs","commentCount":12,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Italy","awayTeamCountryName":"Serbia","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":4,"awayScore":0,"incidents":[{"minute":"11","type":1,"subType":1,"playerName":"Hakan Çalhanoglu","playerId":110373,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"59","type":1,"subType":1,"playerName":"Marko Arnautovic","playerId":34693,"participatingPlayerName":"Mehdi Taremi","participatingPlayerId":358801,"runningScore":null,"field":0,"period":3},{"minute":"71","type":1,"subType":1,"playerName":"Lautaro Martínez","playerId":299344,"participatingPlayerName":"Mehdi Taremi","participatingPlayerId":358801,"runningScore":null,"field":0,"period":3},{"minute":"81","type":1,"subType":2,"playerName":"Mehdi Taremi","playerId":358801,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:31Z","firstHalfEndedAtUtc":"2024-10-01T19:47:36Z","secondHalfStartedAtUtc":"2024-10-01T20:02:38Z"},{"stageId":23663,"id":1866131,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":44,"homeTeamName":"Borussia Dortmund","homeYellowCards":2,"homeRedCards":0,"awayTeamId":103,"awayTeamName":"Celtic","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:37:34Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"de","awayTeamCountryCode":"gb-sct","commentCount":18,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Germany","awayTeamCountryName":"Scotland","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":7,"awayScore":1,"incidents":[{"minute":"7","type":1,"subType":2,"playerName":"Emre Can","playerId":111212,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"9","type":1,"subType":1,"playerName":"Daizen Maeda","playerId":377287,"participatingPlayerName":"Arne Engels","participatingPlayerId":410506,"runningScore":null,"field":1,"period":1},{"minute":"11","type":1,"subType":1,"playerName":"Karim Adeyemi","playerId":392646,"participatingPlayerName":"Julian Brandt","participatingPlayerId":141556,"runningScore":null,"field":0,"period":1},{"minute":"29","type":1,"subType":1,"playerName":"Karim Adeyemi","playerId":392646,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"40","type":1,"subType":2,"playerName":"Serhou Guirassy","playerId":236506,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"42","type":1,"subType":1,"playerName":"Karim Adeyemi","playerId":392646,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"66","type":1,"subType":1,"playerName":"Serhou Guirassy","playerId":236506,"participatingPlayerName":"Nico Schlotterbeck","participatingPlayerId":371275,"runningScore":null,"field":0,"period":3},{"minute":"79","type":1,"subType":1,"playerName":"Felix Nmecha","playerId":367783,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:29Z","firstHalfEndedAtUtc":"2024-10-01T19:46:31Z","secondHalfStartedAtUtc":"2024-10-01T20:03:04Z"},{"stageId":23663,"id":1866137,"status":6,"startTime":"2024-10-01T20:00:00","homeTeamId":65,"homeTeamName":"Barcelona","homeYellowCards":0,"homeRedCards":0,"awayTeamId":587,"awayTeamName":"BSC Young Boys","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-01 21:49:49Z","elapsed":"FT","lastScorer":1,"isTopMatch":true,"homeTeamCountryCode":"es","awayTeamCountryCode":"ch","commentCount":9,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Spain","awayTeamCountryName":"Switzerland","startTimeUtc":"2024-10-01T19:00:00Z","homeScore":5,"awayScore":0,"incidents":[{"minute":"8","type":1,"subType":1,"playerName":"Robert Lewandowski","playerId":29400,"participatingPlayerName":"Raphinha","participatingPlayerId":300447,"runningScore":null,"field":0,"period":1},{"minute":"34","type":1,"subType":1,"playerName":"Raphinha","playerId":300447,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"37","type":1,"subType":1,"playerName":"Iñigo Martínez","playerId":106885,"participatingPlayerName":"Pedri","participatingPlayerId":402197,"runningScore":null,"field":0,"period":1},{"minute":"51","type":1,"subType":1,"playerName":"Robert Lewandowski","playerId":29400,"participatingPlayerName":"Iñigo Martínez","participatingPlayerId":106885,"runningScore":null,"field":0,"period":3},{"minute":"81","type":1,"subType":3,"playerName":"Mohamed Ali Camara","playerId":363637,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-01T19:00:50Z","firstHalfEndedAtUtc":"2024-10-01T19:47:51Z","secondHalfStartedAtUtc":"2024-10-01T20:03:32Z"},{"stageId":23663,"id":1866172,"status":6,"startTime":"2024-10-02T17:45:00","homeTeamId":1706,"homeTeamName":"Shakhtar Donetsk","homeYellowCards":1,"homeRedCards":0,"awayTeamId":300,"awayTeamName":"Atalanta","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 18:49:03Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"ua","awayTeamCountryCode":"it","commentCount":13,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Ukraine","awayTeamCountryName":"Italy","startTimeUtc":"2024-10-02T16:45:00Z","homeScore":0,"awayScore":3,"incidents":[{"minute":"21","type":1,"subType":1,"playerName":"Berat Djimsiti","playerId":100962,"participatingPlayerName":"Ademola Lookman","participatingPlayerId":299451,"runningScore":null,"field":1,"period":1},{"minute":"44","type":1,"subType":1,"playerName":"Ademola Lookman","playerId":299451,"participatingPlayerName":"Sead Kolasinac","participatingPlayerId":110260,"runningScore":null,"field":1,"period":1},{"minute":"48","type":1,"subType":1,"playerName":"Raoul Bellanova","playerId":349184,"participatingPlayerName":"Davide Zappacosta","participatingPlayerId":141312,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T16:45:09Z","firstHalfEndedAtUtc":"2024-10-02T17:30:27Z","secondHalfStartedAtUtc":"2024-10-02T17:46:11Z"},{"stageId":23663,"id":1866208,"status":6,"startTime":"2024-10-02T17:45:00","homeTeamId":2783,"homeTeamName":"Girona","homeYellowCards":6,"homeRedCards":0,"awayTeamId":256,"awayTeamName":"Feyenoord","awayYellowCards":3,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 19:22:29Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"es","awayTeamCountryCode":"nl","commentCount":7,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Spain","awayTeamCountryName":"Netherlands","startTimeUtc":"2024-10-02T16:45:00Z","homeScore":2,"awayScore":3,"incidents":[{"minute":"19","type":1,"subType":1,"playerName":"David López","playerId":92691,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1},{"minute":"23","type":1,"subType":3,"playerName":"Yangel Herrera","playerId":318505,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":1},{"minute":"33","type":1,"subType":1,"playerName":"Antoni Milambo","playerId":422921,"participatingPlayerName":"Igor Paixão","participatingPlayerId":436553,"runningScore":null,"field":1,"period":1},{"minute":"73","type":1,"subType":1,"playerName":"Donny van de Beek","playerId":275035,"participatingPlayerName":"Arnaut Danjuma","participatingPlayerId":323625,"runningScore":null,"field":0,"period":3},{"minute":"79","type":1,"subType":3,"playerName":"Ladislav Krejcí","playerId":404245,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T16:45:51Z","firstHalfEndedAtUtc":"2024-10-02T17:32:59Z","secondHalfStartedAtUtc":"2024-10-02T17:48:25Z"},{"stageId":23663,"id":1866196,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":390,"homeTeamName":"Sturm Graz","homeYellowCards":5,"homeRedCards":0,"awayTeamId":124,"awayTeamName":"Club Bruges","awayYellowCards":1,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 20:23:38Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"at","awayTeamCountryCode":"be","commentCount":1,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Austria","awayTeamCountryName":"Belgium","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":0,"awayScore":1,"incidents":[{"minute":"23","type":1,"subType":1,"playerName":"Christos Tzolis","playerId":399296,"participatingPlayerName":"Raphael Onyedika","participatingPlayerId":425338,"runningScore":null,"field":1,"period":1}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:00:14Z","firstHalfEndedAtUtc":"2024-10-02T19:46:26Z","secondHalfStartedAtUtc":"2024-10-02T20:01:35Z"},{"stageId":23663,"id":1866161,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":299,"homeTeamName":"Benfica","homeYellowCards":1,"homeRedCards":0,"awayTeamId":63,"awayTeamName":"Atletico Madrid","awayYellowCards":4,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 21:46:01Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"pt","awayTeamCountryCode":"es","commentCount":16,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Portugal","awayTeamCountryName":"Spain","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":4,"awayScore":0,"incidents":[{"minute":"13","type":1,"subType":1,"playerName":"Kerem Aktürkoglu","playerId":402880,"participatingPlayerName":"Fredrik Aursnes","participatingPlayerId":303544,"runningScore":null,"field":0,"period":1},{"minute":"52","type":1,"subType":2,"playerName":"Ángel Di María","playerId":23110,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3},{"minute":"75","type":1,"subType":1,"playerName":"Alexander Bah","playerId":401520,"participatingPlayerName":"Jan-Niklas Beste","participatingPlayerId":353193,"runningScore":null,"field":0,"period":3},{"minute":"84","type":1,"subType":2,"playerName":"Orkun Kökçü","playerId":355870,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:02:06Z","firstHalfEndedAtUtc":"2024-10-02T19:50:05Z","secondHalfStartedAtUtc":"2024-10-02T20:06:44Z"},{"stageId":23663,"id":1866218,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":684,"homeTeamName":"Dinamo Zagreb","homeYellowCards":5,"homeRedCards":0,"awayTeamId":248,"awayTeamName":"Monaco","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 21:54:36Z","elapsed":"FT","lastScorer":1,"isTopMatch":false,"homeTeamCountryCode":"hr","awayTeamCountryCode":"fr","commentCount":3,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Croatia","awayTeamCountryName":"France","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":2,"awayScore":2,"incidents":[{"minute":"48","type":1,"subType":1,"playerName":"Petar Sucic","playerId":439534,"participatingPlayerName":"Martin Baturina","participatingPlayerId":407944,"runningScore":null,"field":0,"period":1},{"minute":"66","type":1,"subType":1,"playerName":"Martin Baturina","playerId":407944,"participatingPlayerName":"Marko Rog","participatingPlayerId":203436,"runningScore":null,"field":0,"period":3},{"minute":"74","type":1,"subType":1,"playerName":"Mohammed Salisu","playerId":362357,"participatingPlayerName":"Maghnes Akliouche","participatingPlayerId":426648,"runningScore":null,"field":1,"period":3},{"minute":"90","type":1,"subType":2,"playerName":"Denis Zakaria","playerId":252419,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":null,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:03:21Z","firstHalfEndedAtUtc":"2024-10-02T19:52:29Z","secondHalfStartedAtUtc":"2024-10-02T20:10:22Z"},{"stageId":23663,"id":1866146,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":607,"homeTeamName":"Lille","homeYellowCards":2,"homeRedCards":0,"awayTeamId":52,"awayTeamName":"Real Madrid","awayYellowCards":5,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 20:48:30Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"fr","awayTeamCountryCode":"es","commentCount":20,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"France","awayTeamCountryName":"Spain","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":1,"awayScore":0,"incidents":[{"minute":"47","type":1,"subType":2,"playerName":"Jonathan David","playerId":383855,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":1}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:01:21Z","firstHalfEndedAtUtc":"2024-10-02T19:50:46Z","secondHalfStartedAtUtc":"2024-10-02T20:05:43Z"},{"stageId":23663,"id":1866152,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":26,"homeTeamName":"Liverpool","homeYellowCards":4,"homeRedCards":0,"awayTeamId":71,"awayTeamName":"Bologna","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 21:32:17Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"gb-eng","awayTeamCountryCode":"it","commentCount":22,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"England","awayTeamCountryName":"Italy","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":2,"awayScore":0,"incidents":[{"minute":"11","type":1,"subType":1,"playerName":"Alexis Mac Allister","playerId":345319,"participatingPlayerName":"Mohamed Salah","participatingPlayerId":108226,"runningScore":null,"field":0,"period":1},{"minute":"75","type":1,"subType":1,"playerName":"Mohamed Salah","playerId":108226,"participatingPlayerName":"Dominik Szoboszlai","participatingPlayerId":369875,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:00:12Z","firstHalfEndedAtUtc":"2024-10-02T19:45:56Z","secondHalfStartedAtUtc":"2024-10-02T20:02:19Z"},{"stageId":23663,"id":1866119,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":24,"homeTeamName":"Aston Villa","homeYellowCards":3,"homeRedCards":0,"awayTeamId":37,"awayTeamName":"Bayern Munich","awayYellowCards":2,"awayRedCards":0,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 21:39:15Z","elapsed":"FT","lastScorer":0,"isTopMatch":true,"homeTeamCountryCode":"gb-eng","awayTeamCountryCode":"de","commentCount":50,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"England","awayTeamCountryName":"Germany","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":1,"awayScore":0,"incidents":[{"minute":"79","type":1,"subType":1,"playerName":"Jhon Durán","playerId":433517,"participatingPlayerName":"Pau Torres","participatingPlayerId":344259,"runningScore":null,"field":0,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":0,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:01:29Z","firstHalfEndedAtUtc":"2024-10-02T19:48:36Z","secondHalfStartedAtUtc":"2024-10-02T20:05:15Z"},{"stageId":23663,"id":1866121,"status":6,"startTime":"2024-10-02T20:00:00","homeTeamId":7614,"homeTeamName":"RB Leipzig","homeYellowCards":1,"homeRedCards":0,"awayTeamId":87,"awayTeamName":"Juventus","awayYellowCards":2,"awayRedCards":1,"hasIncidentsSummary":true,"hasPreview":true,"scoreChangedAt":"2024-10-02 21:46:41Z","elapsed":"FT","lastScorer":1,"isTopMatch":true,"homeTeamCountryCode":"de","awayTeamCountryCode":"it","commentCount":8,"isLineupConfirmed":true,"isStreamAvailable":false,"matchIsOpta":false,"homeTeamCountryName":"Germany","awayTeamCountryName":"Italy","startTimeUtc":"2024-10-02T19:00:00Z","homeScore":2,"awayScore":3,"incidents":[{"minute":"30","type":1,"subType":1,"playerName":"Benjamin Sesko","playerId":424191,"participatingPlayerName":"Loïs Openda","participatingPlayerId":363618,"runningScore":null,"field":0,"period":1},{"minute":"50","type":1,"subType":1,"playerName":"Dusan Vlahovic","playerId":315369,"participatingPlayerName":"Andrea Cambiaso","participatingPlayerId":423440,"runningScore":null,"field":1,"period":3},{"minute":"59","type":2,"subType":3,"playerName":"Michele Di Gregorio","playerId":319696,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":1,"period":3},{"minute":"65","type":1,"subType":2,"playerName":"Benjamin Sesko","playerId":424191,"participatingPlayerName":null,"participatingPlayerId":0,"runningScore":null,"field":0,"period":3},{"minute":"68","type":1,"subType":1,"playerName":"Dusan Vlahovic","playerId":315369,"participatingPlayerName":"Francisco Conceição","participatingPlayerId":409221,"runningScore":null,"field":1,"period":3},{"minute":"82","type":1,"subType":1,"playerName":"Francisco Conceição","playerId":409221,"participatingPlayerName":"Nicolò Fagioli","participatingPlayerId":369426,"runningScore":null,"field":1,"period":3}],"bets":null,"aggregateWinnerField":null,"winnerField":1,"period":7,"extraResultField":null,"homeExtratimeScore":null,"awayExtratimeScore":null,"homePenaltyScore":null,"awayPenaltyScore":null,"startedAtUtc":"2024-10-02T19:00:25Z","firstHalfEndedAtUtc":"2024-10-02T19:52:31Z","secondHalfStartedAtUtc":"2024-10-02T20:09:06Z"}]}],"localization":{"match_status_all":"All","match_status_live":"In Play","match_status_upcoming":"Upcoming","livescores_info_refresh":"Refreshing in {0} seconds...","livescores_info_starts_hrs_mins":"Matches start in {0} hours {1} minutes","livescores_info_starts_mins":"Matches start in {0} minutes","livescores_info_loading":"Loading...","favourite_info_please":"Please","favourite_lnk_singin":"Sign In","favourite_lnk_register":"Sign Up","info_today":"Today","lbl_user_odds_display_type":"Odds","sex_abbrv_woman":"W","ls_view_in_mc":"View in Match Centre","lnk_datepicker_view_current_week":"View current week","lnk_datepicker_view_current_month":"View current month ","fltr_livescores_summary_yesterday":"Yesterday","fltr_livescores_summary_today":"Today","fltr_livescores_summary_tomorrow":"Tomorrow","GK":"GK","DR":"DR","DC":"DC","DL":"DL","DMR":"DMR","DMC":"DMC","DML":"DML","MR":"MR","MC":"MC","ML":"ML","AMR":"AMR","AMC":"AMC","AM":"AMC","AML":"AML","FWR":"FWR","FW":"FW","FC":"FW","FWL":"FWL","Sub":"Sub","abd":"Abd","aet":"AET","can":"Can","et1":"ET1","et2":"ET2","ft":"FT","ht":"HT","pen":"PEN","post":"Post","susp":"Susp"},"meta":{"imagesUrlBase":"https://d2zywfiolv4f83.cloudfront.net/img/"},"showOddsToggle":true,"isSummary":false,"oddsToggled":true,"bookmakerCode":"B3","timezoneOffset":60,"locale":"en","calendarType":"Weekly","fixtureDate":"2024-10-01T00:00:00Z"}--></script>